https://towardsdatascience.com/talk-to-your-sql-database-using-langchain-and-azure-openai-bb79ad22c5e2

https://communities.sas.com/t5/SAS-Communities-Library/Exploring-LangChain-and-Azure-OpenAI-s-Ability-to-Write-SQL-and/ta-p/909778

https://python.langchain.com/v0.1/docs/use_cases/sql/agents/#agent

https://python.langchain.com/v0.1/docs/use_cases/sql/



In [1]:
%pip install --upgrade --quiet  langchain langchain-community

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.utilities import SQLDatabase
import os 
from langchain.chat_models import AzureChatOpenAI  
from langchain.agents.agent_toolkits import SQLDatabaseToolkit  
from langchain.agents.agent_types import AgentType  
from langchain.agents import create_sql_agent  
import openai_config  

import warnings
warnings.filterwarnings("ignore")

In [4]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db", sample_rows_in_table_info=3)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [5]:
context = db.get_context()
print(list(context))
print(context["table_info"])

['table_info', 'table_names']

CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("S

In [6]:
context

{'table_info': '\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/*\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/\n\n\nCREATE TABLE "Artist" (\n\t"ArtistId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(120), \n\tPRIMARY KEY ("ArtistId")\n)\n\n/*\n3 rows from Artist table:\nArtistId\tName\n1\tAC/DC\n2\tAccept\n3\tAerosmith\n*/\n\n\nCREATE TABLE "Customer" (\n\t"CustomerId" INTEGER NOT NULL, \n\t"FirstName" NVARCHAR(40) NOT NULL, \n\t"LastName" NVARCHAR(20) NOT NULL, \n\t"Company" NVARCHAR(80), \n\t"Address" NVARCHAR(70), \n\t"City" NVARCHAR(40), \n\t"State" NVARCHAR(40), \n\t"Country" NVARCHAR(40), \n\t"PostalCode" NVARCHAR(10), \n\t"Phone" NVARCHAR(24), \n\t"Fax" NVARCHAR(24), \n\t"Email" NVARCHAR(60) NOT NULL, \n\t"Supp

In [7]:
llm = AzureChatOpenAI(api_key=openai_config.api_key, api_version=openai_config.api_version, azure_endpoint=openai_config.api_base, azure_deployment=openai_config.deployment_id, temperature = 0)

In [8]:
llm.invoke("what is openai?")

AIMessage(content="OpenAI is an artificial intelligence research lab made up of both for-profit and non-profit arms. It was founded in December 2015 by Elon Musk, Sam Altman, and other prominent tech leaders. OpenAI's mission is to ensure that artificial general intelligence (AGI) benefits all of humanity. They aim to build safe and beneficial AGI directly, but are also committed to aiding others in achieving this outcome.", response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 12, 'total_tokens': 96}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6c20e78f-cc93-42f2-85f9-0ac453c80b9c-0')

In [9]:
# Set up SQL toolkit for LangChain Agent  
toolkit = SQLDatabaseToolkit(db=db, llm=llm)  

In [10]:
# Initialize and run the Agent  
agent_executor = create_sql_agent(  
    llm=llm,  
    toolkit=toolkit,  
    verbose=True,  
)

In [11]:
# Ask Questions  
agent_executor.run("how many tables can be joined? by what key?")
# ... Add more questions here ...



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackI need to check the schema of these tables to see which keys they have in common for joining.
Action: sql_db_schema
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId

'11 tables can be joined. The keys for joining are ArtistId, SupportRepId, ReportsTo, CustomerId, InvoiceId, TrackId, AlbumId, MediaTypeId, GenreId, and PlaylistId.'

In [12]:
agent_executor.run("what are the tracks from artist Aerosmith?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe relevant tables for this query seem to be Artist and Track. I should check the schema of these tables to understand their structure and relationships.
Action: sql_db_schema
Action Input: Artist, Track
CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Ge

'The tracks from artist Aerosmith are \'Amazing\', \'Angel\', \'Blind Man\', \'Crazy\', "Cryin\'", \'Deuces Are Wild\', \'Dude (Looks Like A Lady)\', \'Eat The Rich\', "Janie\'s Got A Gun", and "Livin\' On The Edge".'

In [13]:
agent_executor.run("how many tracks composed the artist Aerosmith?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe relevant tables for this question seem to be Artist and Track. I should check the schema of these tables to understand their structure and relationships.
Action: sql_db_schema
Action Input: Artist, Track
CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES 

'Aerosmith composed 15 tracks.'

In [14]:
agent_executor.run("how many tracks are related to the artist Aerosmith?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe relevant tables for this query seem to be Artist and Track. I should check the schema of these tables to understand their structure and relationships.
Action: sql_db_schema
Action Input: Artist, Track
CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Ge

'There are 15 tracks related to the artist Aerosmith.'

In [15]:
agent_executor.run(
    """
    how many tracks are related to the artist Aerosmith?
 
    AlbumId and ArtistId can be used for joining tables
    """
    )



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe relevant tables for this query are likely to be Artist, Album, and Track. I should check the schema of these tables to confirm.
Action: sql_db_schema
Action Input: Artist, Album, Track
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 

'There are 15 tracks related to the artist Aerosmith.'

In [16]:
agent_executor.run(
    """
    how many tracks are related to the artist Aerosmith?
 
    AlbumId and ArtistId can be used for joining tables
    only columns with the same name can be use on joins form different tables
    your sql query have to be a select, changing data is forbiden
    """
    )



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe relevant tables for this query seem to be Artist, Album, and Track. I should check the schema of these tables to understand their structure and relationships.
Action: sql_db_schema
Action Input: Artist, Album, Track
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	

'There are 15 tracks related to the artist Aerosmith.'

In [17]:
agent_executor.run(
    """
    
    # Question
    How many tracks are related to the artist Aerosmith?
    
    # Context

    Generated query will run in sqlite database

    Artist table has information about a band that write songs and create albums.
    Album table contains information about albums written by an artist. This table can be joined with the table Artist using column ArtistId.
    Track table has information about a song. This table can be joined with the table Album using column AlbumId.
    
    Only columns with the same name can be use on joins from different tables.
    Your result sql query can only be a select, insert, update, or delete words are strongly forbidden.
    """
)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe relevant tables for this query are Artist, Album, and Track. I should check the schema of these tables to understand their structure and relationships.
Action: sql_db_schema
Action Input: "Artist, Album, Track" 
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Nam

'There are 15 tracks related to the artist Aerosmith.'

In [18]:
agent_executor.run("how many tracks are?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe 'Track' table seems to be the most relevant to the question. I should check its schema to understand its structure.
Action: sql_db_schema
Action Input: Track
CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), 
	FOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")
)

/*
3 rows from Track table:
TrackId	Name	AlbumId	MediaTypeId	GenreId	Composer	Milliseconds	Bytes	UnitPrice
1	For Those About To Rock (We Salute You)	1	

'There are 3503 tracks.'

In [19]:
agent_executor.run("Describe the playlisttrack table")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe PlaylistTrack table is in the database. I should query the schema of the PlaylistTrack table.
Action: sql_db_schema
Action Input: "PlaylistTrack"
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/I now know the final answer
Final Answer: The "PlaylistTrack" table has two columns: "PlaylistId" and "TrackId". Both are integers and are not null. They form the primary key of the table. The "TrackId" column references the "TrackId" in the "Track" table, and the "PlaylistId" column references the "PlaylistId" in the 

'The "PlaylistTrack" table has two columns: "PlaylistId" and "TrackId". Both are integers and are not null. They form the primary key of the table. The "TrackId" column references the "TrackId" in the "Track" table, and the "PlaylistId" column references the "PlaylistId" in the "Playlist" table. Here are some sample rows from the "PlaylistTrack" table: \n1. PlaylistId: 1, TrackId: 3402\n2. PlaylistId: 1, TrackId: 3389\n3. PlaylistId: 1, TrackId: 3390'

# Example: describing a table, recovering from an error

In [20]:
agent_executor.run("Describe the playlistsong table")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe "PlaylistTrack" table seems to be the most relevant to the question. I should query the schema of this table.
Action: sql_db_schema
Action Input: "PlaylistTrack"
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/I now know the final answer
Final Answer: The "PlaylistTrack" table has two columns: "PlaylistId" and "TrackId". Both are integers and are not null. They form the primary key of the table. The "TrackId" column references the "TrackId" in the "Track" table, and the "PlaylistId" column references the "Pla

'The "PlaylistTrack" table has two columns: "PlaylistId" and "TrackId". Both are integers and are not null. They form the primary key of the table. The "TrackId" column references the "TrackId" in the "Track" table, and the "PlaylistId" column references the "PlaylistId" in the "Playlist" table.'

# Example: running queries

In [21]:
agent_executor.run("List the total sales per country. Which country's customers spent the most?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe relevant tables for this query seem to be the Customer and Invoice tables. I should check their schemas to see what columns they have.
Action: sql_db_schema
Action Input: Customer, Invoice
CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	S

'The country with the highest total sales is the USA with a total of $523.06.'

In [22]:
agent_executor.run("Show the total number of tracks in each playlist. The Playlist name should be included in the result.")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe relevant tables for this query seem to be Playlist and PlaylistTrack. I will now check the schema of these tables to understand their structure and the relationships between them.
Action: sql_db_schema
Action Input: Playlist, PlaylistTrack
CREATE TABLE "Playlist" (
	"PlaylistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("PlaylistId")
)

/*
3 rows from Playlist table:
PlaylistId	Name
1	Music
2	Movies
3	TV Shows
*/


CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/The Playlist table has a column 

"Here are the total number of tracks in each playlist:\n- '90’s Music': 1477 tracks\n- 'Brazilian Music': 39 tracks\n- 'Classical': 75 tracks\n- 'Classical 101 - Deep Cuts': 25 tracks\n- 'Classical 101 - Next Steps': 25 tracks\n- 'Classical 101 - The Basics': 25 tracks\n- 'Grunge': 15 tracks\n- 'Heavy Metal Classic': 26 tracks\n- 'Music': 6580 tracks\n- 'Music Videos': 1 track\n- 'On-The-Go 1': 1 track\n- 'TV Shows': 426 tracks"

# Recovering from an error
In this example, the agent is able to recover from an error after initially trying to access an attribute (Track.ArtistId) which doesn’t exist.

In [23]:
agent_executor.run("Who are the top 3 best selling artists?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe relevant tables for this query seem to be Artist, Album, and InvoiceLine. I will need to check the schema of these tables to understand their structure and relationships.
Action: sql_db_schema
Action Input: Artist, Album, InvoiceLine
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "InvoiceLine" (
	"Invoi

'The top 3 best selling artists are Iron Maiden, U2, and Metallica.'

In [24]:
agent_executor("Which Artist recorded the longest track?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe relevant tables for this query seem to be the Artist and Track tables. I should check their schemas to see what columns they have.
Action: sql_db_schema
Action Input: Artist, Track
CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), 
	

{'input': 'Which Artist recorded the longest track?',
 'output': "'Battlestar Galactica' recorded the longest track."}

In [25]:
agent_executor.run("what's the name of the person that made the most expensive purchase")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe relevant tables for this query seem to be the Customer and Invoice tables. I should check their schemas to see what information they contain.
Action: sql_db_schema
Action Input: Customer, Invoice
CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	

'The person who made the most expensive purchase is Helena Holý.'

In [26]:
agent_executor.run("how many tables are in the db?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackI now know the final answer
Final Answer: There are 11 tables in the database.

> Finished chain.


'There are 11 tables in the database.'